# Traditional Methods trained on every file, tested on one at a time (untuned)
MinMaxScaler is applied to these tests. All models are untuned. The model will be trained on file A, tested on files B,C,D.., then on B, tested on A,C,D.. etc

In [1]:
import sys
sys.path.append("../../")
from time import process_time
from os import listdir, chdir, environ

def warn(*args, **kwargs):
  pass
import warnings
warnings.warn = warn

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    environ["PYTHONWARNINGS"] = "ignore"

import numpy as np
from modules.NetworkTraffic import NT2
from sklearn import model_selection, metrics

from autosklearn.classification import AutoSklearnClassifier

FilesToTest = list()
chdir("../../data")
for file in listdir():
  if file.endswith(".csv"):
    FilesToTest.append(file)

TestSize = [0.4]
model = AutoSklearnClassifier(time_left_for_this_task=30, memory_limit=4096, n_jobs=-1)
OutputResults = dict()

In [2]:
theseFiles = dict()
for file in FilesToTest:
  theseFiles[file] = NT2(file, transform=False, drop=True)

In [3]:
from copy import deepcopy

OutputResults.clear()

for file in theseFiles:
  print(file, end=', ')
  OutputResults[file] = dict()
  currentFileData = theseFiles[file]
  restOfFiles = deepcopy(theseFiles)
  restOfFiles.pop(file)
  print('[', end='')
  OutputResults[file]["ASKL"] = dict()
  model.fit(currentFileData.data, currentFileData.target)

  for file2 in restOfFiles:
    print(file2, end=' ')
    testFileData = theseFiles[file2]
    y_pred = model.predict(testFileData.data)
    score = metrics.accuracy_score(testFileData.target, y_pred)
    OutputResults[file]["ASKL"].update({file2: {"Accuracy":score}})
  print(']')

b5000d100.csv, [b5000d30.csv b100d10.csv b1000d10.csv b1000d100.csv b100d100.csv b5000d10.csv b1000d30.csv b100d30.csv ]
b5000d30.csv, [b5000d100.csv b100d10.csv b1000d10.csv b1000d100.csv b100d100.csv b5000d10.csv b1000d30.csv b100d30.csv ]
b100d10.csv, [b5000d100.csv b5000d30.csv b1000d10.csv b1000d100.csv b100d100.csv b5000d10.csv b1000d30.csv b100d30.csv ]
b1000d10.csv, [b5000d100.csv b5000d30.csv b100d10.csv b1000d100.csv b100d100.csv b5000d10.csv b1000d30.csv b100d30.csv ]
b1000d100.csv, [b5000d100.csv b5000d30.csv b100d10.csv b1000d10.csv b100d100.csv b5000d10.csv b1000d30.csv b100d30.csv ]
b100d100.csv, [b5000d100.csv b5000d30.csv b100d10.csv b1000d10.csv b1000d100.csv b5000d10.csv b1000d30.csv b100d30.csv ]
b5000d10.csv, [b5000d100.csv b5000d30.csv b100d10.csv b1000d10.csv b1000d100.csv b100d100.csv b1000d30.csv b100d30.csv ]
b1000d30.csv, [b5000d100.csv b5000d30.csv b100d10.csv b1000d10.csv b1000d100.csv b100d100.csv b5000d10.csv b100d30.csv ]
b100d30.csv, [b5000d100.csv b500

In [4]:
import json
with open("EveryFileTransfer_Untuned_AllTestResults.json", "w") as f:
  f.write(json.dumps(OutputResults, indent=2))

In [5]:
with open("EveryFileTransfer_Untuned_ModelResults.csv", "w") as f3:
  f3.write("Trained On,Tested On,Model,Accuracy\n")
  for file in OutputResults:
    for file2 in OutputResults[file]:
      for model in OutputResults[file][file2]:
        f3.write(f"{file},{model},{file2},{OutputResults[file][file2][model]['Accuracy']}\n")